In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from datetime import datetime, timedelta

import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau

sns.set_style('whitegrid')
plt.style.use("fivethirtyeight")
%matplotlib inline

from google.colab import drive
drive.mount('/content/drive')

#SBER_PATH = '/content/drive/MyDrive/Stock_Data/SBER_D_from_15_11_20_to_11_12_23.csv'
SBER_PATH = '/content/drive/MyDrive/Stock_Data/sber.csv'

def fix_stockdata(ticket):
    for i in range(len(ticket['Цена'])):

        price = ticket['Цена'].iloc[i]
        price = price.replace(',', '.')
        price = price.replace('.', '', price.count('.') - 1)
        ticket['Цена'].iloc[i] = price

        open = ticket['Откр.'].iloc[i]
        open = open.replace(',', '.')
        open = open.replace('.', '', open.count('.') - 1)
        ticket['Откр.'].iloc[i] = open

        max = ticket['Макс.'].iloc[i]
        max = max.replace(',', '.')
        max = max.replace('.', '', max.count('.') - 1)
        ticket['Макс.'].iloc[i] = max

        min = ticket['Мин.'].iloc[i]
        min = min.replace(',', '.')
        min = min.replace('.', '', min.count('.') - 1)
        ticket['Мин.'].iloc[i] = min

        volume = ticket['Объём'].iloc[i]
        volume = volume.replace(',', '.')
        volume = volume.replace('.', '', volume.count('.') - 1)
        if volume[len(volume) - 1] == 'K':
            volume = int(float(volume[:len(volume) - 1]) * 10**3)
        elif volume[len(volume) - 1] == 'M':
            volume = int(float(volume[:len(volume) - 1]) * 10**6)
        elif volume[len(volume) - 1] == 'B':
            volume = int(float(volume[:len(volume) - 1]) * 10**9)
        else:
            volume = int(float(volume))
        ticket['Объём'].iloc[i] = volume

        diff = ticket['Изм. %'].iloc[i]
        diff = diff.replace(',', '.')
        diff = float(diff[:len(diff) - 1])
        ticket['Изм. %'].iloc[i] = diff

    ticket['Цена'] = ticket['Цена'].astype(float)
    ticket['Откр.'] = ticket['Откр.'].astype(float)
    ticket['Макс.'] = ticket['Макс.'].astype(float)
    ticket['Мин.'] = ticket['Мин.'].astype(float)
    ticket['Объём'] = ticket['Объём'].astype(int)
    ticket['Изм. %'] = ticket['Изм. %'].astype(float)
    ticket['Дата'] = pd.to_datetime(ticket['Дата'], dayfirst=True).dt.normalize()

    ticket = ticket.set_index('Дата')

    return ticket

DATA = fix_stockdata(pd.read_csv('sber_19-24.csv'))
DATA = DATA.rename(columns={'Дата' : 'date', 'Цена' : 'close', 'Откр.' : 'open', 'Макс.' : 'max', 'Мин.' : 'min', 'Объём' : 'volume', 'Изм. %' : 'return'})
DATA.index.rename('date', inplace=True)
DATA = DATA.sort_values(by='date', ascending=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


FileNotFoundError: [Errno 2] No such file or directory: 'sber_19-24.csv'

In [ ]:
def raw_reader(stock):
  COMMENTS = '/content/drive/MyDrive/Social_Media_Data/sber_raw/investing_dataset_'+stock.lower()+'_1-201.csv'

  storage_path = '/content/drive/MyDrive/Social_Media_Data/'+stock.lower()+'_raw/'
  SMD = pd.read_csv(storage_path + 'investing_dataset_'+stock.lower()+'_1-201.csv')
  for f in os.listdir(storage_path):
    if f != 'investing_dataset_'+stock.lower()+'_1-201.csv':
      SMD = pd.concat([SMD, pd.read_csv(storage_path + f)], ignore_index=True)

  SMD.drop(columns=SMD.columns[0], axis=1, inplace=True)
  SMD['date'] = pd.to_datetime(SMD['date'], dayfirst=False).dt.normalize()
  SMD['user_id'] = SMD['user_id'].astype(str)
  for i in range(len(SMD['user_id'])):
          id = SMD['user_id'].iloc[i]
          if len(id) == 11:
            id = id[:len(id)-2]
            SMD['user_id'].iloc[i] = id

  SMD.to_csv('/content/drive/MyDrive/Social_Media_Data/'+stock.lower()+'_investing_dataset.csv')
  print(stock.upper(), 'successfully saved')



def read_SMD(stock):
  SMD = pd.read_csv('/content/drive/MyDrive/Social_Media_Data/'+stock.lower()+'_investing_dataset.csv')
  SMD.drop(columns=SMD.columns[0], axis=1, inplace=True)
  SMD['date'] = pd.to_datetime(SMD['date'], dayfirst=False).dt.normalize()
  SMD['user_id'] = SMD['user_id'].astype(str)
  return SMD



if 'sber' + '_investing_dataset.csv' in os.listdir('/content/drive/MyDrive/Social_Media_Data/'):
  print('sber', 'dataset already exists')
else:
  raw_reader('sber')

sber dataset already exists


In [ ]:
DATA_SMD_new = pd.read_csv('/content/drive/MyDrive/Social_Media_Data/sber_investing_dataset.csv')

In [ ]:
DATA_SMD_old = pd.read_csv('/content/drive/MyDrive/Social_Media_Data/sber_investing_dataset_old.csv')

In [ ]:
DATA_SMD_new

,Unnamed: 0,user_id,date,comment,likes,dislikes
0,0,235766022.0,2024-06-03,"Если меньше тракториста уйдет, паника будет",2,1
1,1,225645875.0,2024-06-03,"Я все таки надеюсь удержат минимум, сломают к ...",0,0
2,2,240209431.0,2024-06-03,"Если хомяки на панике начнут сливать, то всё, ...",3,1
3,3,225645875.0,2024-06-03,"Повторюсь, ниже 304.35(минимума прошлого месяц...",2,3
4,4,251161925.0,2024-06-03,ну вот опять я в полной попе сижу (,0,0
...,...,...,...,...,...,...
16104,16104,238919612.0,2023-12-05,по 120 к теперь гробы стоят,0,1
16105,16105,250380648.0,2023-12-05,Втб 😢 следующий спербанк,5,8
16106,16106,202717967.0,2023-12-05,За время наблюдения увидел вот что- Мистер Тан...,1,2
16107,16107,255062676.0,2023-12-05,&quot;Торговля на тандыре&quot; - новая страте...,5,1


In [ ]:
DATA = pd.concat((DATA_SMD_new, DATA_SMD_old)).drop('Unnamed: 0', axis=1).reset_index(drop=True)

In [ ]:
DATA.to_csv('/content/drive/MyDrive/Social_Media_Data/sber_investing_dataset.csv')

NameError: name 'DATA' is not defined

In [ ]:
DATA = pd.read_csv('/content/drive/MyDrive/Stock_Data/sber_data_complete.csv')

In [ ]:
DATA.index = DATA['date']
#DATA = DATA.sort_values(by='date', ascending=True)

In [ ]:
DATA

,date,close,open,max,min,volume,return,comments,sentiment_sum,positive,negative
date,,,,,,,,,,,
2024-06-03,2024-06-03,308.15,313.50,315.35,305.01,52980000,-1.58,68.0,-17.795575,20,48
2024-05-31,2024-05-31,313.11,316.00,318.44,309.84,49210000,-1.11,274.0,-38.080781,115,159
2024-05-30,2024-05-30,316.63,320.91,321.54,316.00,27010000,-1.17,151.0,-25.633767,58,93
2024-05-29,2024-05-29,320.38,318.25,320.40,315.94,22990000,0.68,110.0,-30.876960,33,77
2024-05-28,2024-05-28,318.22,317.50,320.90,315.89,30190000,0.36,131.0,-16.074570,59,72
...,...,...,...,...,...,...,...,...,...,...,...
2020-11-09,2020-11-09,228.05,220.00,231.30,218.70,217540000,4.85,1037.0,-111.691360,443,594
2020-11-06,2020-11-06,217.50,215.80,218.29,213.00,95000000,0.37,831.0,-129.184823,321,510
2020-11-05,2020-11-05,216.70,214.40,218.65,211.43,152550000,3.59,1208.0,-136.892515,508,700


In [ ]:
def sentiment_price(ticket):
  vol_price = pd.concat([ticket['close'], ticket['sentiment_sum']], axis=1)
  vol_price['close'] = (vol_price['close'] - vol_price['close'].median()) / (max(vol_price['close']) - min(vol_price['close']))
  vol_price['sentiment_sum'] = (vol_price['sentiment_sum'] - vol_price['sentiment_sum'].median()) / (max(vol_price['sentiment_sum']) - min(vol_price['sentiment_sum']))
  fig = px.line(vol_price, hover_data=[ticket['close'], ticket['sentiment_sum']])
  fig.show()

sentiment_price(DATA['2022-02-27':'2021-10-21'])

In [ ]:
px.line(DATA[['close', 'sentiment_sum']])

In [ ]:
index = 38
print(DATA.loc[index]['comment'])
get_sentiment(DATA.loc[index]['comment'], 'score')

3156 пробили....


-0.6733654849231243

In [ ]:
DATA_SMD_new = DATA_SMD_new[:16109]

In [ ]:
DATA_SMD_old

,user_id,date,comment,likes,dislikes
0,205075820,2023-12-05,Греф нам уже обещал сбер перед СВО и чем в ито...,1,0
1,239380774,2023-12-05,"Я продал сбер, хватит, с 250 заходил. Теперь л...",1,1
2,234517670,2023-12-05,Ребят...узбагойтсь! Все ещё будет. Есть ощущен...,1,0
3,205075820,2023-12-05,Втб льют конечно жёстко. Полагаю со сбером так...,1,0
4,243325716,2023-12-05,Круги Ганна сжались в преддверье дня грефа,1,0
...,...,...,...,...,...
356832,209289540,2020-11-02,и все выкупили,1,0
356833,217268107,2020-11-02,"204,15 шорт",2,0
356834,216477429,2020-11-02,"На МосБиржу работает, закрывает по 0.25 )).. ...",1,1
356835,214586249,2020-11-02,А кто серии постил на счет шотрящих хомячков.,0,0


In [ ]:
!pip install transformers sentencepiece --quiet
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
model_checkpoint = 'cointegrated/rubert-tiny-sentiment-balanced'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)
if torch.cuda.is_available():
    model.cuda()

def get_sentiment(text, return_type='label'):
    """ Calculate sentiment of a text. `return_type` can be 'label', 'score' or 'proba' """
    with torch.no_grad():
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(model.device)
        proba = torch.sigmoid(model(**inputs).logits).cpu().numpy()[0]
    if return_type == 'label':
        return model.config.id2label[proba.argmax()]
    elif return_type == 'score':
        return proba.dot([-1, 0, 1])
    return proba

text = 'Какая гадость эта ваша заливная рыба!'
# classify the text
print(get_sentiment(text, 'label'))  # negative
# score the text on the scale from -1 (very negative) to +1 (very positive)
print(get_sentiment(text, 'score'))  # -0.5894946306943893
# calculate probabilities of all labels
print(get_sentiment(text, 'proba'))  # [0.7870447  0.4947824  0.19755007]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/377 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/241k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/468k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/884 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/47.2M [00:00<?, ?B/s]

negative
-0.5894947499036789
[0.78704476 0.4947824  0.19755001]


In [ ]:
DATA_SMD = DATA_SMD_new

In [ ]:
DATA_SMD['sentiment'] = DATA_SMD['comment'].apply(lambda x: get_sentiment(x, 'score')) # !

In [ ]:
DATA_SMD = pd.read_csv('sber_smd_sentiment.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'sber_smd_sentiment.csv'

In [ ]:
DATA_SMD_new['comment'] = DATA_SMD_new['comment'].astype(str) # !

In [ ]:
#DATA_SMD.to_csv('sber_smd_sentiment.csv')

In [ ]:
DATA_SMD['label'] = DATA_SMD['sentiment'].apply(lambda x: 1 if x > 0 else -1) # !

In [ ]:
DATA_SMD_new = DATA_SMD[:]

In [ ]:
Q = DATA_SMD['date'].value_counts() # !
N = pd.DataFrame([[date, Q[date],
                   DATA_SMD[DATA_SMD['date'] == date]['sentiment'].sum()] for date in Q.keys()], columns=['date', 'comments', 'sentiment_sum'])
N = N.set_index('date')
N = N.sort_values(by='date', ascending=False)

In [ ]:
DATA.index = DATA.index.astype('str') # !

In [ ]:
DATA = DATA.join(N)
DATA = DATA.dropna()

In [ ]:
positive = []
negative = []
for date in DATA.index:
  if -1 in DATA_SMD[DATA_SMD['date'] == date]['label'].value_counts().index:
    negative.append(DATA_SMD[DATA_SMD['date'] == date]['label'].value_counts()[-1])
  else:
    negative.append(0)
  if 1 in DATA_SMD[DATA_SMD['date'] == date]['label'].value_counts().index:
    positive.append(DATA_SMD[DATA_SMD['date'] == date]['label'].value_counts()[1])
  else:
    positive.append(0)

In [ ]:
DATA['positive'] = positive
DATA['negative'] = negative

In [ ]:
DATA.to_csv('sber_data_complete.csv')

In [ ]:
DATA = DATA.read_csv('sber_data_complete.csv')

In [ ]:
test = DATA[['return', 'sentiment_sum']]
test['sentiment_sum'] = StandardScaler().fit_transform(test['sentiment_sum'].values.reshape(-1, 1)).reshape(-1)
px.line(test)